In [1]:
from fileinput import filename
import os
import random

import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from utils import settings as settings
import pandas as pd
from evaluation.metrics import accuracy
from evaluation.conf_matrix import create_conf_matrix
from loader.Preprocessor import Preprocessor
from models.JensModel import JensModel
from models.RainbowModel import RainbowModel
from models.ResNetModel import ResNetModel
from utils.filter_activities import filter_activities
from utils.folder_operations import new_saved_experiment_folder
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig
from tensorflow.keras.layers import (Dense)
from utils.Recording import Recording
import matplotlib.pyplot as plt

2022-05-23 21:41:12.164772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 21:41:12.164797: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Init
# TODO: refactor, find out why confusion matrix sometimes shows different results than accuracy
# TODO: - make train and test datasets evenly distributed
#       - make 
""" 
Number of recordings per person
{'connie.csv': 6, 'alex.csv': 38, 'trapp.csv': 9, 'anja.csv': 13, 'aileen.csv': 52, 'florian.csv': 16, 'brueggemann.csv': 36, 'oli.csv': 20, 'rauche.csv': 9, 'b2.csv': 6, 'yvan.csv': 8, 'christine.csv': 7, 'mathias.csv': 2, 'kathi.csv': 17}
"""

data_config = LabPoseConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data_new_preprocessed')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
#data_config = SonarConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data_alex')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
settings.init(data_config)
random.seed(1678978086101)

k_fold_splits = 3
numEpochsBeforeTL = 10
numEpochsForTL = 3
minimumRecordingsPerLeftOutPerson = 5
# Load dataset
recordings = settings.DATA_CONFIG.load_dataset()#limit=75


Loading recording 0_marco.csv, 1 / 36
22
Loading recording 1_felix.csv, 2 / 36
22
Loading recording 2_valentin.csv, 3 / 36
22
Loading recording 3_orhan.csv, 4 / 36
22
Loading recording 4_orhan.csv, 5 / 36
22
Loading recording 5_orhan.csv, 6 / 36
22
Loading recording 6_orhan.csv, 7 / 36
22
Loading recording 7_orhan.csv, 8 / 36
22
Loading recording 8_felix.csv, 9 / 36
22
Loading recording 9_kirill.csv, 10 / 36
22
Loading recording 10_daniel.csv, 11 / 36
22
Loading recording 11_valentin.csv, 12 / 36
22
Loading recording 12_alex.csv, 13 / 36
22
Loading recording 13_daniel.csv, 14 / 36
22
Loading recording 14_lucas.csv, 15 / 36
22
Loading recording 15_orhan.csv, 16 / 36
22
Loading recording 16_felix.csv, 17 / 36
22
Loading recording 17_orhan.csv, 18 / 36
22
Loading recording 18_tobi.csv, 19 / 36
22
Loading recording 19_lucas.csv, 20 / 36
22
Loading recording 20_felix .csv, 21 / 36
22
Loading recording 21_lucas.csv, 22 / 36
22
Loading recording 22_daniel.csv, 23 / 36
22
Loading recording 23_

In [9]:

import pose_sequence_loader
import importlib
importlib.reload(pose_sequence_loader)

from pose_sequence_loader import deleteConsecutives, getPoseFilesWithMetadata

dataset_path = '/dhc/groups/bp2021ba1/data/lab_data'
pose_file_metadata_tuples = getPoseFilesWithMetadata(dataset_path, settings.DATA_CONFIG.category_labels)

#print(pose_file_metadata_tuples[0][1])

def findPoseFile(recording_activities, pose_file_metadata_tuples):
    for pose_file, pose_activites, startTimestamp in pose_file_metadata_tuples:
        if len(recording_activities) != len(pose_activites):
            continue
        for i, recording_activity in enumerate(recording_activities):
            if recording_activity != pose_activites[i] and pose_activites[i] != -1:
                continue
            if i == len(recording_activities) - 1: 
                return pose_file, startTimestamp

    return None, None

for recording in recordings:
    activity_list = deleteConsecutives(recording.activities)

    pose_file_name, startTimestamp = findPoseFile(activity_list, pose_file_metadata_tuples)
    print(pose_file_name)
    #print(activity_list)

# for k in pose_file_metadata_tuples:
#     print(k[1])

    

[22, 4, 2, 22, 6, 5, 22, 11, 22, 11, 22, 6, 14, 22, 15, 22, 1, 15, 1, 6, 22, 9, 1, 22, 1, 22, 1, 6, 22, 6, 0, 22, 6, 16, 22, 0, 22, 9, 1, 22, 2, 4, 22]
[22, 2, 5, 23, 2, 5, 2, 14, 6, 22, 12, 22, 0, 12, 4, 22]
[22, 4, 2, 6, 5]
[22, 1, 22, 1, 22, 1, 22, 1, 22, 16, 22]
[22, 4, 2, 22, 5, 11, 22, 2, 22, 0, 22, 14, 22, 12, 4, 16, 22, 2]
[22, 4, 2, 22, 5]
[22, 4, 22, 2, 22, 5, 22, 5, 22, 2, 22, 5, 22, 5, 22, 2, 22, 4, 22, 6, 22, 4, 22, 1, 22, 1, 22, 1, 22, 6, 22, 0, 12, 22, 0, 22, 14, 22, 12, 6, 22, 16, 22]
[22, 11]
[22, 6, 22, 2, 5, 2, 12, 0, 22, 5, 6, 11, 22, 2, 6, 14, 22, 0, 22, 12, 4]
[22, 6, 5, 4, 22, 2, 6, 22, 9, 22, 1, 22, 5, 6, 19, 1, 22, 6, 11, 6, 22, 2, 14, 22, 2, 22, 4, 6, 22, 17, 22, 17, 22, 6, 22]
[22, 4, 22, 4, 22, 2, 22, 5, 22, 5, 22, 2, 22, 2, 22, 5, 22, 5, 22, 2, 22, 0, 22, 2, 22, 1, 22, 6, 22, 0, 22, 12, 22, 4, 22]
[22, 4, 2, 5, 2, 22, 14, 22, 12, 22, 0, 22, 2, 12, 22, 16, 6, 17, 22, 4]
[22, 4, 22, 2, 22, 6, 22, 5, 22, 5, 2, 22, 2, 22, 5, 22, 11, 22, 11, 22, 6, 22, 16, 22, 1